In [1]:
import numpy as np
import seaborn as sn
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from keras.models import Model, Sequential
from keras.applications.nasnet import NASNetLarge
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam

In [2]:
train_datagen = ImageDataGenerator( rescale = 1./255,
                                    validation_split=0.2,
                                    rotation_range = 30,
                                    width_shift_range = 0.2,
                                    height_shift_range = 0.2,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True,
                                    vertical_flip=True,
                                    fill_mode = 'nearest'
                                  )

val_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [3]:
train_data = train_datagen.flow_from_directory('data/train',
                                                target_size = (48,48),
                                                class_mode = 'categorical',
                                                batch_size = 64,
                                                subset='training'
                                              )

val_data = val_datagen.flow_from_directory('data/train',
                                            target_size = (48,48),
                                            class_mode = 'categorical',
                                            batch_size = 64,
                                            subset='validation'
                                          )

test_data = test_datagen.flow_from_directory('data/test',
                                              target_size = (48,48),
                                              class_mode = 'categorical',
                                              batch_size = 64
                                            )

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
train_steps = train_data.n//train_data.batch_size
val_steps = val_data.n//val_data.batch_size
test_steps = test_data.n//test_data.batch_size

In [5]:
# VGG16 model
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(48, 48, 3))

# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False
    
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # Assuming 7 classes for emotions

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

In [6]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
checkpoint = ModelCheckpoint('emotion_model.h5', save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training the model
history = model.fit(train_data,
                    steps_per_epoch=train_steps,
                    epochs=50,
                    validation_data=val_data,
                    validation_steps=val_steps,
                    callbacks=[reduce_lr, checkpoint, early_stop]
                   )

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_data, steps=test_steps)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')



Epoch 1/50
358/358 [==============================] - ETA: 0s - loss: 1.9799 - accuracy: 0.2487

C:\Users\samia\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


358/358 [==============================] - 314s 875ms/step - loss: 1.9799 - accuracy: 0.2487 - val_loss: 1.6966 - val_accuracy: 0.3183 - lr: 0.0010
Epoch 2/50
358/358 [==============================] - 307s 858ms/step - loss: 1.7684 - accuracy: 0.2840 - val_loss: 1.6876 - val_accuracy: 0.3281 - lr: 0.0010
Epoch 3/50
358/358 [==============================] - 312s 871ms/step - loss: 1.7219 - accuracy: 0.3067 - val_loss: 1.6548 - val_accuracy: 0.3406 - lr: 0.0010
Epoch 4/50
358/358 [==============================] - 304s 849ms/step - loss: 1.7048 - accuracy: 0.3101 - val_loss: 1.6566 - val_accuracy: 0.3425 - lr: 0.0010
Epoch 5/50
358/358 [==============================] - 303s 847ms/step - loss: 1.7015 - accuracy: 0.3178 - val_loss: 1.6485 - val_accuracy: 0.3439 - lr: 0.0010
Epoch 6/50
358/358 [==============================] - 308s 860ms/step - loss: 1.6847 - accuracy: 0.3241 - val_loss: 1.6651 - val_accuracy: 0.3478 - lr: 0.0010
Epoch 7/50
358/358 [==============================] - 307